In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# import plotting as plot
# import itertools
# import pyreporter as pr
import utility

import recon
recon3d = recon.recon.Recon()
util = recon.utilities.Utilities()

sns.set_style('white')
sns.set_style('ticks')

In [ ]:
rna = pd.read_csv('../../data/NAD_cell_lines/gene_fpkm.xls.csv', index_col=0)
rna = rna[rna.columns[rna.columns.str.contains('wtHEK293|HEK25a51|HEK25A51')]]
rna = rna[~(rna == 0).any(axis=1)]
rna = util.map_gene(df=rna, g_mapping=recon3d.genes, mapping_column='ensembl_gene')
rna_wt = rna.filter(like='wtHEK')
rna_ko = rna.filter(like='HEK25a51ko')
rna_oe = rna.filter(like='HEK25A51oe')
rna = pd.concat((rna_wt.mean(axis=1), rna_ko.mean(axis=1), rna_oe.mean(axis=1)), axis=1)
rna.columns = ['wt', 'ko', 'oe']

In [ ]:
rdict = (rna['ko']-(rna['wt'])).div(rna['wt'], axis=0).replace(np.inf, np.nan).dropna()
rdict.head()

In [ ]:
res = pd.read_csv('../../data/NAD_cell_lines/pr_slc25a51ko_wt.csv', index_col=0, sep='\t')
# res = util.reshape(res, include=None)

In [ ]:
# gs = util.reshape(recon3d.gs, include=None)
gs = recon3d.gs

gnames = recon3d.genes.set_index('gene_number').to_dict()['symbol']
gnames = {str(k):v for k,v in gnames.items()}

In [ ]:
mets = pd.read_csv('../../data/NAD_cell_lines/merged_metabolites_measurement.csv')
_gs = util.reshape(gs.T.mul(rdict, axis=0).dropna().T, include=None)
gs_met = _gs.query('metabolites in @mets.fullName.unique() and compartment != "extracellular"')
gs_met = gs_met.groupby('metabolites').mean()

gs_met = gs_met.rename(columns=gnames)
gs_met.head()

In [ ]:
rdict.max()

In [ ]:
sns.set_context('talk', font_scale=0.6)

fig, ax = plt.subplots(figsize=(1., 5))
cm = sns.heatmap(data=pd.DataFrame(rdict, columns=['transcriptomics compared to baseline']), cmap='seismic', 
                 center=0, robust=True, xticklabels=False, yticklabels=False, 
                 ax=ax, cbar_kws={'label': '$\Delta\Phi$', 'location':'top'},
                )
cm.set_ylabel('genes')
fig.savefig('../../images/NAD_cell_lines/SLC25A51ko/differential_rnaseq.png', bbox_inches='tight')

In [ ]:
_gs

In [ ]:
sns.set_context('talk', font_scale=0.8)

cm = sns.clustermap(data=_gs.groupby('metabolites').mean(), cmap='seismic', center=0, robust=True, 
                figsize=(6,5), yticklabels=False, dendrogram_ratio=(0.15, 0.0), method='ward',
                xticklabels=False, cbar_kws={'label': '$C_{\Phi}^{M}$'}, cbar_pos=(0.02, 0.8, 0.02, 0.2),
                col_cluster=False, row_cluster=False,
                )
cm.ax_heatmap.set_xlabel('genes')
cm.ax_heatmap.set_ylabel('metabolites')
cm.savefig('../../images/NAD_cell_lines/SLC25A51ko/response_coefficients_all.png', bbox_inches='tight')

In [ ]:
consistent = ['2,3-Diphosphoglyceric acid', '6-Phosphogluconic acid', 'ATP',
       'Citric acid', 'D-Glucose', 'D-Sedoheptulose 7-phosphate',
       'Fructose 6-phosphate', 'Glucose 1-phosphate', 'Glucose 6-phosphate',
       'Isocitric acid', 'L-Alanine', 'L-Arginine', 'L-Aspartic acid',
       'L-Glutamine', 'L-Lactic acid', 'L-Lysine', 'L-Phenylalanine',
       'L-Proline', 'L-Threonine', 'L-Tryptophan', 'L-Tyrosine',
       'Pyruvic acid', 'Succinic acid', 'cis-Aconitic acid']

In [ ]:

sns.set_context('talk', font_scale=0.6)
_gs_met = gs_met.T[(gs_met.T.abs() > 0.005).any(axis=1)].T
# _gs_met = (_gs_met - _gs_met.mean()).div(_gs_met.std(axis=1), axis=0)

# fig, ax = plt.subplots(figsize=(20, 6))
cm = sns.clustermap(data=_gs_met, cmap='seismic', center=0, robust=True, 
                col_cluster=False, row_cluster=False,
                figsize=(15, 8), yticklabels=True, dendrogram_ratio=(0.12, 0.0), method='weighted',
                xticklabels=True, cbar_kws={'label': '$\Gamma^{\Phi}_{M}$'}, cbar_pos=(0.02, 0.8, 0.02, 0.18),
                )
cm.ax_heatmap.set_xlabel('genes', fontdict={'size': 20})
cm.ax_heatmap.set_ylabel('metabolites', fontdict={'size': 20})

cm.savefig('../../images/NAD_cell_lines/SLC25A51ko/response_coefficients.svg', bbox_inches='tight')

In [ ]:
sns.set_context('talk', font_scale=0.6)
_gs_met = gs_met.T[(gs_met.T.abs() > 0.005).any(axis=1)].T

cm = sns.clustermap(data=_gs_met[_gs_met.index.isin(consistent)], cmap='seismic', center=0, robust=True, 
                col_cluster=False, row_cluster=False,
                figsize=(15, 5), yticklabels=True, dendrogram_ratio=(0.12, 0.0), method='weighted',
                xticklabels=True, cbar_kws={'label': '$\Gamma^{\Phi}_{M}$'}, cbar_pos=(0.02, 0.8, 0.02, 0.18),
                )
cm.ax_heatmap.set_xlabel('genes', fontdict={'size': 20})
cm.ax_heatmap.set_ylabel('metabolites', fontdict={'size': 20})
cm.savefig('../../images/NAD_cell_lines/SLC25A51ko/response_coefficients_tol005_consistent.svg', bbox_inches='tight')

In [ ]:
rdict

In [ ]:
mets = pd.read_csv('../../data/NAD_cell_lines/merged_metabolites_measurement.csv')
m_wt = mets[mets.group == 'CTRL'].pivot_table(index='fullName', columns='replicate', values='Amount/mg Protein')
m_ko = mets[mets.group == 'KO'].pivot_table(index='fullName', columns='replicate', values='Amount/mg Protein')
m_oe = mets[mets.group == 'OE'].pivot_table(index='fullName', columns='replicate', values='Amount/mg Protein')
m_wt.columns = ['wt1', 'wt2', 'wt3', 'wt4', 'wt5']
m_ko.columns = ['ko1', 'ko2', 'ko3', 'ko4', 'ko5']
m_oe.columns = ['oe1', 'oe2', 'oe3', 'oe4', 'oe5']

In [ ]:
y_gs = _gs[_gs.metabolites.isin(mets.fullName.unique())].groupby('metabolites').mean().sum(axis=1)
x_gs = _gs[_gs.metabolites.isin(mets.fullName.unique())].groupby('metabolites').mean()[rdict.index].sum(axis=1)
m_ko_wt = (m_ko.mean(axis=1)  - m_wt.mean(axis=1)).div(m_wt.mean(axis=1), axis=0).replace(np.inf, np.nan).dropna()

print(np.corrcoef(m_ko_wt, x_gs)[0,1])
print(np.corrcoef(x_gs, y_gs)[0,1])

sns.scatterplot(x=x_gs, y=y_gs, color='k', alpha=0.5)
sns.scatterplot(x=m_ko_wt, y=y_gs, color='r', alpha=0.5)


In [ ]:
_df = pd.concat((m_ko_wt, y_gs), axis=1)
_df.columns = ['ko_wt', 'gs']
_df[(_df < 0.0).all(axis=1)|(_df > 0.0).all(axis=1)]

In [ ]:
def get_genes_from_metabolite(met):
    _rxns = recon3d.get_reactions_from_metabolite(met)
    _rxns_gpr = {k:v for k,v in recon3d.gpr.items() if k in _rxns.abbreviation.values}
    _rxns_genes = utility.get_genenumbers_from_rxn(_rxns_gpr)
    return _rxns_genes

def calculate_alpha(met):
    _rxns_genes = get_genes_from_metabolite(met)
    _gs_met = gs[(gs.metabolites == met) & (gs.compartment != 'extracellular')].groupby('metabolites').mean()[_rxns_genes]
    _alpha_met = (_gs_met[_gs_met.columns.intersection(rdict.index)].abs().sum(axis=1)).div(_gs_met.abs().sum(axis=1), axis=0).iloc[0]
    return _alpha_met

In [ ]:
for met in mets.fullName.unique():
    print(met, calculate_alpha(met))

In [ ]:
for met in mets.fullName.unique():
    _rxns_genes = get_genes_from_metabolite(met)
    _gs_met = gs[(gs.metabolites == met) & (gs.compartment != 'extracellular')].groupby('metabolites').mean()[_rxns_genes]
    _c_met = _gs_met[_gs_met.columns.intersection(rdict.index)]
    _df1 = pd.concat((np.log2(rdict[rdict.index.isin(_rxns_genes)]), _c_met.sum()), axis=1)
    _df1.columns = ['expression', 'control']
    print(met, _df1.corr().iloc[0,1], calculate_alpha(met))

In [ ]:
_df1[(_df1 > 0.0).all(axis=1) | (_df1 < 0.0).all(axis=1)].corr()
_df1[~((_df1 > 0.0).all(axis=1) | (_df1 < 0.0).all(axis=1))]#.corr()

In [ ]:
gnames = recon3d.genes.set_index('gene_number').to_dict()['symbol']
gnames = {str(k):v for k,v in gnames.items()}

gs_mets = gs.query('metabolites in @mets.fullName.unique() and compartment != "extracellular"').groupby('metabolites').mean()

# rg = []
# for met in mets.fullName.unique():
#     _rxns_genes = get_genes_from_metabolite(met)
#     rg = rg + _rxns_genes
# rg = list(set(rg))
# gs_mets = gs_mets[gs_mets.columns.intersection((rdict.index).intersection(rg))]
# gs_mets = gs_mets.rename(columns=gnames).T

In [ ]:
gs

In [ ]:
fig, ax = plt.subplots(figsize=(25,8))
sns.heatmap(data=gs_mets.T, cmap='RdBu_r', center=0.0, vmin=-0.1, vmax=0.1,
            yticklabels=True, ax=ax, cbar_kws={'label': 'centrality control coefficient'})

In [ ]:
gs_mets

In [ ]:
sns.set_context('paper', font_scale=1.2)
tol = 1e-2
_gs_mets = gs_mets[(gs_mets > tol).any(axis=1) | (gs_mets < -tol).all(axis=1)]
_gs_mets_norm = (_gs_mets - _gs_mets.mean()).div(_gs_mets.std(axis=1), axis=0)

# _gs_mets_norm
cm = sns.clustermap(data=_gs_mets_norm.T, cmap='RdBu_r', center=0.0, figsize=(20,10), #vmin=-0.2, vmax=0.2, 
               col_cluster=True, row_cluster=True, yticklabels=True, xticklabels=True, dendrogram_ratio=(0.1, 0.2),
               cbar_kws={'label': 'normalised coefficient'})
cm.ax_cbar.set_position((0.02, 0.8, 0.02, 0.2))
cm.figure.savefig('../../images/NAD_cell_lines/SLC25A51ko/control_coefficients_tol.png', bbox_inches='tight')
# fig, ax = plt.subplots(figsize=(25,8))
# sns.heatmap(data=_gs_mets_norm.T, cmap='RdBu_r', center=0.0, #vmin=-0.01, vmax=0.01,
#             yticklabels=True, xticklabels=True, ax=ax,
#             cbar_kws={'label': 'normalised centrality control coefficient'})

In [ ]:
rdf = pd.DataFrame(rdict, columns=['ko/wt'])
rdf.index = rdf.index.map(gnames)

In [ ]:
rdf

In [ ]:
fig, ax = plt.subplots(figsize=(25,2))
sns.heatmap(data=np.log2(rdf[rdf.index.isin(_gs_mets_norm.index)].T), cmap='RdBu_r', center=0.0, ax=ax, cbar_kws={'label': 'log2(expression change)'})

In [ ]:
for met in mets.fullName.unique():
    sdict = gs.query(f'metabolites == "{met}"').groupby('metabolites').mean().T[met]
    _df1 = pd.concat((sdict[sdict.index.isin(rdict.index)].abs(), abs(1-rdict)), axis=1)
    print(met, _df1.corr(method='spearman').iloc[0, 1])

In [ ]:
for met in mets.fullName.unique():
    # print(met)
    rxns = recon3d.get_reactions_from_metabolite(met)
    rxns_gpr = list(set(rxns.abbreviation).intersection(set(recon3d.gpr.keys())))
    mapped_gpr = {k:v for k,v in recon3d.gpr.items() if k in rxns_gpr}
    genes = utility.get_genenumbers_from_rxn(gpr_dict=mapped_gpr)
    sdict = gs.query(f'metabolites == "{met}"').groupby('metabolites').mean().T[met]
    _df1 = pd.concat((sdict[sdict.index.isin(rdict.index)].abs(), abs(1-rdict)), axis=1)
    print(met, _df1[_df1.index.isin(genes)].corr(method='spearman').iloc[0,1])

In [ ]:
_df1[_df1.index.isin(genes)].sort_values('Control', ascending=False).corr(method='spearman').iloc[0,1]

In [ ]:
sns.scatterplot(data= _df1[_df1.index.isin(genes)], x='Control', y='Expression')

In [ ]:
import scipy.stats as stats

def eval_gpr(gpr, model, weights):
    """
    Evaluate GPRs with weights
    :param model: cobra model
    :type model: cobra.Model
    :param weights: weights
    :type weights: pd.DataFrame
    :return: values
    :rtype: pd.Series
    """
    values = {}
    for i in gpr.keys():
        try:
            values[i] = eval(utility.replace_and(gpr[i], weights))
        except SyntaxError:
            values[i] = 1.0
    return values

In [ ]:

values = eval_gpr(mapped_gpr, recon3d.model, sdict)
values_df = eval_gpr(mapped_gpr, recon3d.model, rdict)

In [ ]:
# sns.scatterplot(x=np.log2(list(values_df.values())), y=np.log2(list(values.values())))
values

In [ ]:
assert len(values_df) == len(values)
values_df